# DAVID mk20 - 검정선 기준 좌우 분리 및 버튼 세로 배치

In [ ]:
import ipywidgets as widgets
from IPython.display import display
from datetime import datetime
import json
import os

todo_items = []


In [ ]:
layout_label = widgets.Layout(width='70px')
layout_input = widgets.Layout(width='240px')
layout_button = widgets.Layout(width='110px', height='36px')
layout_wide_button = widgets.Layout(width='240px', height='36px')
layout_progress_bar = widgets.Layout(width='180px', height='16px')

overall_progress_bar = widgets.FloatProgress(min=0, max=100, value=0, bar_style='info', layout=layout_progress_bar)
overall_progress_label = widgets.Label(value="(0%)")

todo_list_box = widgets.VBox()


In [ ]:
def update_overall_progress():
    if not todo_items:
        overall_progress_bar.value = 0
        overall_progress_label.value = "(0%)"
        return
    avg = sum(i['progress'] for i in todo_items) / len(todo_items)
    overall_progress_bar.value = avg
    overall_progress_label.value = f"({int(avg)}%)"


In [ ]:
def refresh_list():
    rows = []
    for item in todo_items:
        # 좌측 항목 라벨들
        label_col = widgets.VBox([
            widgets.Label("할 일", layout=layout_label),
            widgets.Label("진행률", layout=layout_label),
            widgets.Label(" ", layout=layout_label),  # 슬라이더 간격
            widgets.Label("입력 마감일", layout=layout_label),
            widgets.Label("중요도", layout=layout_label),
            widgets.Label("작업", layout=layout_label),
        ])

        # 우측 항목 내용들
        title = widgets.Text(value=item['title'], layout=layout_input)
        bar = widgets.FloatProgress(value=item['progress'], min=0, max=100, style={'bar_color': 'blue'}, layout=layout_progress_bar)
        slider = widgets.IntSlider(value=item['progress'], min=0, max=100, step=5, layout=layout_input)
        due = widgets.Label(value=item['due'].strftime('%Y-%m-%d') if item['due'] else "-", layout=layout_input)
        priority = widgets.Label(value=item['priority'], layout=layout_input)

        complete_btn = widgets.Button(description="완료✅", layout=layout_button)
        delete_btn = widgets.Button(description="삭제❌", layout=layout_button)
        buttons = widgets.VBox([complete_btn, delete_btn])

        content_col = widgets.VBox([title, bar, slider, due, priority, buttons])
        row = widgets.HBox([label_col, content_col], layout=widgets.Layout(border='1px solid black', padding='10px'))
        rows.append(row)

        def on_slider_change(change, i=item, pbar=bar):
            i['progress'] = change['new']
            pbar.value = change['new']
            update_overall_progress()

        def on_complete(b, i=item):
            i['progress'] = 100
            update_overall_progress()
            refresh_list()

        def on_delete(b, i=item):
            todo_items.remove(i)
            update_overall_progress()
            refresh_list()

        slider.observe(on_slider_change, names='value')
        complete_btn.on_click(on_complete)
        delete_btn.on_click(on_delete)

    todo_list_box.children = rows


In [ ]:
title_input = widgets.Text(placeholder="할 일을 입력하세요", layout=layout_input)
progress_input = widgets.IntSlider(min=0, max=100, step=5, layout=layout_input)
due_input = widgets.DatePicker(layout=layout_input)
priority_input = widgets.Dropdown(options=['낮음', '보통', '높음'], value='보통', layout=layout_input)

add_button = widgets.Button(description="추가", button_style='success', layout=layout_wide_button)
save_button = widgets.Button(description="저장", button_style='info', layout=layout_button)
load_button = widgets.Button(description="불러오기", button_style='warning', layout=layout_button)

def add_task(b):
    if not title_input.value.strip():
        return
    todo_items.append({
        'title': title_input.value.strip(),
        'progress': progress_input.value,
        'due': due_input.value,
        'priority': priority_input.value
    })
    title_input.value = ''
    progress_input.value = 0
    due_input.value = None
    priority_input.value = '보통'
    update_overall_progress()
    refresh_list()

add_button.on_click(add_task)


In [ ]:
def save_data(b):
    with open('todo_data.json', 'w') as f:
        json.dump([
            {
                **i,
                'due': i['due'].strftime('%Y-%m-%d') if i['due'] else None
            } for i in todo_items
        ], f)

def load_data(b):
    try:
        if os.path.exists('todo_data.json'):
            with open('todo_data.json', 'r') as f:
                raw = json.load(f)
                todo_items.clear()
                for i in raw:
                    todo_items.append({
                        'title': i['title'],
                        'progress': i['progress'],
                        'due': datetime.strptime(i['due'], '%Y-%m-%d').date() if i['due'] else None,
                        'priority': i.get('priority', '보통')
                    })
                update_overall_progress()
                refresh_list()
    except Exception as e:
        print("불러오기 실패:", e)

save_button.on_click(save_data)
load_button.on_click(load_data)


In [ ]:
header = widgets.Label(value="DAVID mk20 - 좌우 분리 + 완료/삭제 버튼 세로 정렬")

form = widgets.VBox([
    header,
    widgets.HBox([widgets.Label("할 일", layout=layout_label), title_input]),
    widgets.HBox([widgets.Label("진행률", layout=layout_label), progress_input]),
    widgets.HBox([widgets.Label("마감일", layout=layout_label), due_input]),
    widgets.HBox([widgets.Label("중요도", layout=layout_label), priority_input]),
    widgets.HBox([add_button], layout=widgets.Layout(justify_content='center')),
    widgets.HBox([save_button, load_button], layout=widgets.Layout(justify_content='center')),
    widgets.HBox([overall_progress_bar, overall_progress_label], layout=widgets.Layout(justify_content='center')),
    todo_list_box
])

display(form)
update_overall_progress()
refresh_list()
